In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# 데이터 불러오기
data = pd.read_csv(r'/병합상권분석_생존율 추가.csv', encoding='CP949')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Columns: 127 entries, 기준_년분기_코드 to 버스_정거장_수
dtypes: float64(38), int64(86), object(3)
memory usage: 282.9+ KB


In [13]:
# LabelEncoder를 객체로 생성
encoder = LabelEncoder()

data['data_label_encoder1'] = encoder.fit_transform(data['상권_변화_지표'])
data['data_label_encoder2'] = encoder.fit_transform(data['상권_변화_지표_명'])

# feature, target 분리
feature = data.drop(columns=['1년 생존율','3년 생존율','5년 생존율','행정동_코드_명','상권_변화_지표','상권_변화_지표_명'],axis=1)
target = data['5년 생존율']

# train, test 분리
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size = 0.3, random_state = 2024)

# SimpleImputer를 사용하여 결측치 대체
imputer = SimpleImputer(strategy='mean')  # 평균값으로 대체
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)


In [14]:
print(feature)

     기준_년분기_코드    행정동_코드  총_유동인구_수  남성_유동인구_수  여성_유동인구_수  연령대_10_유동인구_수  \
0        20191  11215710   9661988    4729200    4932788         974665   
1        20192  11215710   6448214    3096230    3351985         635168   
2        20193  11215710   8977322    4320908    4656415         875109   
3        20194  11215710   9852808    4736330    5116478         963331   
4        20201  11215710   9347518    4508585    4838932         885366   
..         ...       ...       ...        ...        ...            ...   
280      20223  11215810   6038228    2718420    3319807        1650798   
281      20224  11215810   6160261    2768492    3391769        1716596   
282      20231  11215810   6094173    2747990    3346183        1697648   
283      20232  11215810   6141386    2777246    3364140        1771717   
284      20233  11215810   5968625    2699871    3268754        1734152   

     연령대_20_유동인구_수  연령대_30_유동인구_수  연령대_40_유동인구_수  연령대_50_유동인구_수  ...  슈퍼마켓_수  \
0          3943298 

In [15]:
from sklearn.model_selection import GridSearchCV

# alpha 값 후보 리스트
alphas = np.logspace(-4, 4, 10)

# GridSearchCV를 사용하여 최적의 alpha 값 찾기
lasso_cv = Lasso()
param_grid = {'alpha': alphas}
grid_search = GridSearchCV(lasso_cv, param_grid, cv=5)
grid_search.fit(X_train_imputed, Y_train)

# 최적의 alpha 값
best_alpha = grid_search.best_params_['alpha']
print("최적의 alpha:", best_alpha)

# 최적의 alpha 값으로 라쏘 모델 구축
lasso = Lasso(alpha=best_alpha, max_iter=1000).fit(X_train_imputed, Y_train)

# 회귀계수가 0이 아닌 특성의 인덱스를 찾음
non_zero_coef_indices = [i for i, coef in enumerate(lasso.coef_) if coef != 0]

# 인덱스를 사용하여 원래 feature의 이름을 가져옴
feature_list = feature.columns[non_zero_coef_indices]

print("선택된 특성:", feature_list)

print(lasso.coef_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.091e+03, tolerance: 5.369e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.384e+02, tolerance: 4.848e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.992e+02, tolerance: 5.181e

최적의 alpha: 10000.0
선택된 특성: Index(['총_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수',
       '연령대_60_이상_유동인구_수', '시간대_00_06_유동인구_수', '월_평균_소득_금액', '지출_총금액',
       '식료품_지출_총금액', '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액',
       '교육_지출_총금액', '유흥_지출_총금액', '여가_문화_지출_총금액', '기타_지출_총금액', '음식_지출_총금액',
       '아파트_평균_시가'],
      dtype='object')
[ 0.00000000e+00 -0.00000000e+00 -1.48164343e-06 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.04705332e-06  5.97370447e-06
  2.89527278e-06  0.00000000e+00 -4.64061399e-06  7.27914514e-06
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.963e+03, tolerance: 5.353e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.808e+03, tolerance: 4.723e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.467e+03, tolerance: 6.369e